In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
LEARNING_RATE = 0.14
EPOCHS = 100
TARGET_FOLDER = "weights"
K_FOLDS = 5

In [26]:
import torch.nn as nn
import torch
from torchvision import datasets, transforms
from sklearn.model_selection import KFold

Resize Images to 244, 244. By using to_tensor the images are already normalized between 0 and 1. "The image object is an array of (244, 244, 3) should be flattened to be list (178, 608)." What? wie?

In [ ]:
transform = transforms.Compose([transforms.Resize((256, 256)),
                                 transforms.ToTensor()])

data_set = datasets.ImageFolder('/content/drive/MyDrive/data_aml/X-Ray Image DataSet', transform=transform)


In [ ]:
data_loader = torch.utils.data.DataLoader(data_set, batch_size=32, shuffle=True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Current device: {device}")

Current device: cpu


In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super(ConvBlock, self).__init__()
        for_pad = lambda s: s if s > 2 else 3
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=(for_pad(kernel_size) - 1)//2, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.LeakyReLU(negative_slope=0.1, inplace=True) 

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        return out

class TripleConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TripleConvBlock, self).__init__()
        self.conv_block_1 = ConvBlock(in_channels, out_channels)
        self.conv_block_2 = ConvBlock(out_channels, in_channels, kernel_size=1)  
        self.conv_block_3 = ConvBlock(in_channels, out_channels)

    def forward(self, x):
        out = self.conv_block_1(x)
        out = self.conv_block_2(out)
        out = self.conv_block_3(out)
        return out

class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.seq = nn.Sequential(
        ConvBlock(3, 8),
        nn.MaxPool2d(2, stride=2),
        ConvBlock(8, 16),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(16, 32),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(32,64),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(64,128),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(128,256),
        ConvBlock(256, 128, kernel_size=1),
        ConvBlock(128, 256),
        nn.Conv2d(256, 2, 3, padding=(3-1)//2, stride=1), # Ich bekomme es nicht hin, dass das identisch ist 
        nn.ReLU(),
        nn.BatchNorm2d(2),
        nn.Flatten(),
        nn.Linear(338,2)
        )

    def forward(self, x):
        return self.seq(x)

class Model3(nn.Module):
    def __init__(self):
        super(Model3, self).__init__()
        self.seq = nn.Sequential(
        ConvBlock(3, 8),
        nn.MaxPool2d(2, stride=2),
        ConvBlock(8, 16),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(16, 32),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(32,64),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(64,128),
        nn.MaxPool2d(2, stride=2),
        TripleConvBlock(128,256),
        ConvBlock(256, 128, kernel_size=1),
        ConvBlock(128, 256),
        nn.Conv2d(256, 3, 3, padding=(3-1)//2, stride=1), # Ich bekomme es nicht hin, dass das identisch ist 
        nn.ReLU(),
        nn.BatchNorm2d(3),
        nn.Flatten(),
        nn.Linear(507,3)
        )

    def forward(self, x):
        return self.seq(x)

In [ ]:
from torchsummary import summary
model = Model3()
summary(model, (3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 256, 256]             216
       BatchNorm2d-2          [-1, 8, 256, 256]              16
         LeakyReLU-3          [-1, 8, 256, 256]               0
         ConvBlock-4          [-1, 8, 256, 256]               0
         MaxPool2d-5          [-1, 8, 128, 128]               0
            Conv2d-6         [-1, 16, 128, 128]           1,152
       BatchNorm2d-7         [-1, 16, 128, 128]              32
         LeakyReLU-8         [-1, 16, 128, 128]               0
         ConvBlock-9         [-1, 16, 128, 128]               0
        MaxPool2d-10           [-1, 16, 64, 64]               0
           Conv2d-11           [-1, 32, 64, 64]           4,608
      BatchNorm2d-12           [-1, 32, 64, 64]              64
        LeakyReLU-13           [-1, 32, 64, 64]               0
        ConvBlock-14           [-1, 32,

In [32]:
def train(model, data_loader, optimizer):
    """
    model -- neural net
    data_loader -- dataloader for train images
    optimizer -- optimizer
    """
    model.train()
    
    criterion = torch.nn.CrossEntropyLoss().to(device)
    
    for step, [images, labels] in enumerate(data_loader,0):
        images = images.to(device)
        labels = labels.to(device)
                
        optimizer.zero_grad()
        result = model(images)
        loss = criterion(result, labels)
                
        # backpropagation
        loss.backward()
        optimizer.step()
                                    
        if step%10 == 0:
            print(f"Step: {step}, loss: {loss}")

In [43]:
def calc_accuracy(result, labels):
    result = torch.round(result)
    probs = torch.softmax(result, dim=1)

    correct_results_sum = (probs.argmax(dim=1) == labels).sum().float()
    acc = correct_results_sum/labels.shape[0]
    acc = torch.round(acc * 100)
    
    return acc


In [ ]:
def test(model, test_loader, epoch):
    """    
    model -- neural net 
    test_loader -- dataloader of test images
    epoch -- current epoch
    """
    model.eval()
    loss = 0
    accuracy = 0
    criterion = torch.nn.CrossEntropyLoss().to(device)

    for step, [images, labels] in enumerate(test_loader,0):
        images = images.to(device)
        labels = labels.to(device)

        result = model(images)
        loss += criterion(result.detach(), labels.detach())
        accuracy += calc_accuracy(result.detach(), labels.detach())
    loss /= step
    accuracy /=  step
  
    print(f"Loss: {loss}, Accuracy: {accuracy}")

In [ ]:
kfold = KFold(n_splits=K_FOLDS, shuffle=True)

In [33]:
for fold, (train_ids, test_ids) in enumerate(kfold.split(data_set)):
    
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    train_loader = torch.utils.data.DataLoader(
                      data_set, 
                      batch_size=32, sampler=train_subsampler)
    test_loader = torch.utils.data.DataLoader(
                      data_set,
                      batch_size=32, sampler=test_subsampler)
    
    model = Model3()
    
    optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
    
    for epoch in range(0, EPOCHS):

      train(model, train_loader, optimizer)
      
      torch.save(model.state_dict(), f'{TARGET_FOLDER}/fold_{fold}_epoch_{epoch}.ckpt')

      test(model, test_loader, epoch)

FOLD 0
--------------------------------
Step: 0, loss: 1.3826792240142822


KeyboardInterrupt: ignored

# Federated
https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines/

Neuste Version (0.3.x) hat kein TorchHook -> Lösung finden?

In [ ]:
!pip install syft==0.2.9 

     |████████████████████████████████| 433 kB 14.0 MB/s 
     |████████████████████████████████| 200 kB 64.1 MB/s 
     |████████████████████████████████| 1.8 MB 52.2 MB/s 
     |████████████████████████████████| 126 kB 68.1 MB/s 
     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 57 kB 4.6 MB/s 
     |████████████████████████████████| 2.0 MB 61.1 MB/s 
     |████████████████████████████████| 449 kB 73.0 MB/s 
     |████████████████████████████████| 66 kB 3.9 MB/s 
     |████████████████████████████████| 484 kB 34.6 MB/s 
     |████████████████████████████████| 4.0 MB 62.5 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 753.4 MB 7.9 kB/s 
     |████████████████████████████████| 9.0 MB 57.6 MB/s 
     |████████████████████████████████| 79 kB 6.4 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
     |███████████████████

In [ ]:
import syft as sy


In [ ]:
hook = sy.TorchHook(torch)
nr_of_instances = 2
instances = []
for i in range(nr_of_instances):
  instances.append(sy.VirtualWorker(hook, id=str(i)))

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 32
        self.test_batch_size = 1000
        self.epochs = 100
        self.lr = 0.14
        self.seed = 1
        self.log_interval = 10
        self.save_model = False
args = Arguments()

torch.manual_seed(args.seed)
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

In [ ]:
federated_data_loader = sy.FederatedDataLoader(data_set.federate(instances),
    batch_size=args.batch_size, shuffle=True, **kwargs)

In [35]:
def train(args, model, train_loader, optims, epoch):
    model.train()

    criterion = torch.nn.CrossEntropyLoss().to(device)

    for batch_idx, (data, target) in enumerate(train_loader): # <-- now it is a distributed dataset
        model.send(data.location)
        data, target = data.to(device), target.to(device)

        optimizer = optims.get_optim(data.location.id)
        optimizer.zero_grad()
        
        output = model(data)

        loss = criterion(output, target)

        loss.backward()
        optimizer.step()

        model.get()
        if batch_idx % args.log_interval == 0:
            loss = loss.get()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [ ]:

from syft.federated.floptimizer import Optims



for fold, (train_ids, test_ids) in enumerate(kfold.split(data_set)):
    
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    train_loader = torch.utils.data.DataLoader(
                      data_set, 
                      batch_size=32, sampler=train_subsampler)
    test_loader = torch.utils.data.DataLoader(
                      data_set,
                      batch_size=32, sampler=test_subsampler)
    
    federated_model = Model3()
    
    federated_optimizer = Optims(list(range(nr_of_instances)), optim=torch.optim.Adam(params=federated_model.parameters(),lr=args.lr))
    
    for epoch in range(1, args.epochs + 1):

      train(args, federated_model, federated_data_loader, federated_optimizer, epoch)

      if (args.save_model):
        torch.save(federated_model.state_dict(), f'{TARGET_FOLDER}/federated_fold_{fold}_epoch_{epoch}.ckpt')

      test(federated_model, test_loader, epoch)






   



FOLD 0
--------------------------------
Train Epoch: 1 [0/1152 (0%)]	Loss: 1.585826
Train Epoch: 1 [320/1152 (28%)]	Loss: 0.462757
Train Epoch: 1 [640/1152 (56%)]	Loss: 1.332489
Train Epoch: 1 [960/1152 (83%)]	Loss: 4.794581
Loss: 4139.73193359375, Accuracy: 46.71428680419922
Train Epoch: 2 [0/1152 (0%)]	Loss: 37.647072
Train Epoch: 2 [320/1152 (28%)]	Loss: 8.295449
Train Epoch: 2 [640/1152 (56%)]	Loss: 45.125534
Train Epoch: 2 [960/1152 (83%)]	Loss: 1.111818
Loss: 63.211883544921875, Accuracy: 46.57143020629883
Train Epoch: 3 [0/1152 (0%)]	Loss: 67.502190
Train Epoch: 3 [320/1152 (28%)]	Loss: 12.189524
Train Epoch: 3 [640/1152 (56%)]	Loss: 192.262924
Train Epoch: 3 [960/1152 (83%)]	Loss: 14.047342
Loss: 133.20265197753906, Accuracy: 55.42856979370117
Train Epoch: 4 [0/1152 (0%)]	Loss: 222.663315
Train Epoch: 4 [320/1152 (28%)]	Loss: 2.321252
Train Epoch: 4 [640/1152 (56%)]	Loss: 495.440765
Train Epoch: 4 [960/1152 (83%)]	Loss: 25.120312
Loss: 56.788597106933594, Accuracy: 43.285713195